**Text Mining in R**

# Text, Strings und Reguläre Ausdrücke

Jan R. Riebling

## Agenda

* What is text?
* Text in R
* Text I/O
* String operations

## Zusätzliche Pakete

R-base verfügt aus historischen Gründen nur über sehr eingeschränkte Möglichkeiten Zeichenketten (*strings*) zu verarbeiten. Daher sind für Text Mining zusätzliche Pakete empfohlen:

* `tidyverse`: Meta-Paket für Data Science enthält unter anderem `dplyr`, `stringr` und andere. Die Dokumentation befindet sich [hier]().
* `stringr`: Implementiert Stringfunktionalitäten auf Basis von C Bibliotheken.
* `stringi`: Ausführlichere Version von `stringr`.
* `tidytext`: Zusätzliche Unterstützung für textbasierte Datenstrukturen im Rahmen der `tidyverse` Philosophie. Ausführliche Dokumentation im Buch [„Text Mining with R“]().
* `tm`: Stellt Datenstrukturen zum Ugang mit Textkorpora bereit. Siehe hierzu die entsprechende [Dokumentation](https://cran.r-project.org/web/packages/tm/vignettes/tm.pdf).

In [4]:
install.packages(c('tidytext', 'tm'))

Installiere Pakete nach ‘/home/jrriebling/.local/lib/R’
(da ‘lib’ nicht spezifiziert)



In [5]:
library(tidyverse)
library(stringr)
library(tm)

Lade nötiges Paket: NLP


Attache Paket: ‘NLP’


Das folgende Objekt ist maskiert ‘package:ggplot2’:

    annotate




## Tibbles vs. data.frames

For most purposes, a `tibble`-object works almost the same as a `data.frame`. In fact, the `tibble` is much more strict and exhibits the following behaviors:

* Never converts or transforms data types, variable names or assigns row names automatically.
* No autocompletion of variable names.
* Subsetting with the index notation `[` always returns a `tibble`-object.

# Maschinenlesbarer Text

## A formal definition of language

A language $\mathcal{L}$ can be defined as consisting of an alphabet $\Sigma{}$ and a grammar containing the rules for the construction of valid expressions. The alphabet of a language consists of a set of "words" $w$, the smallest elements of a language. Under the model of a generative grammar the construction of a valid sentence can be described as a set of substitution rules, starting from the level of the "sentence" and stopping when the word level is reached. If each substitution is correct the resulting sentence is correct. Only formal languages can be described entirely in terms of a grammar.

## Grammar 

> Eine Grammatik $G$ ist ein Tupel $G = (V, T, R, S)$. Dabei ist
> * $V$ eine endliche Menge von Variablen;
> * $T$ eine endliche Menge von Terminalen; es gilt $V \cap T = \emptyset$;
> * $R$ eine endliche Menge von Regeln. Eine Regel ist ein Element $(P, Q)$ aus
> $(V \cup T )∗ V (V \cup T )∗ \times (V \cup T )∗$ . Das heißt, P ist ein Wort über $(V \cup T)$,
> das mindestens eine Variable aus $V$ enthält, während $Q$ ein beliebiges Wort über $(V \cup T)$ ist. P heißt auch Prämisse und Q Konklusion > der Regel. Für eine Regel (P, Q) ∈ R schreibt man üblicherweise u auch P →G Q
> oder nur P → Q.
> * S das Startsymbol, S ∈ V .

(Erk & Priese 2008, 54)


## Text als "String"

Standardmäßig wird Text in Computern als eine Zeichenkette definiert. Die Grammatik die einen validen String erzeugt wird als eine reguläre Grammatik: Vom einem Starpunkt werden Elemente in eine vordefinierte Richtung hinzugefügt bis ein Endsymbol erreicht wird.

In Basis-R werden Strings mittels der Klasse `character` repräsentiert.

In [2]:
class("This is a string!")

[1] "character"

# Eigenschaften von Strings

## Definition

Strings werden entweder mit `"` oder `'` geöffnet und müssen mit dem selben Zeichen geschlossen werden.

In [15]:
string1 <- "This is a string."
string2 <- 'This is another one, using single quotes'
string1

[1] "This is a string."

## Sonderzeichen

In [13]:
"This will work"

[1] "This will work"

In [14]:
'But this won't'

ERROR: Error in parse(text = x, srcfile = src): <text>:1:15: unerwartetes Symbol
1: 'But this won't
                  ^


## Escaping

Um Sonderzeichen in Strings zu repräsentieren oder die Funktion als Sonderzeichen zu unterdrücken müssen diese „escaped“ werden. Dies geschieht durch Voranstellen von `\`, hat jedoch zur Folge das eine Repräsentation von `\` im String selbst so auszusehen hat: `\\`. 

Sonderzeichen:

* `\n`: die (UNIX) newline; Ein Zielenumbruch.
* `\r`: Carriage Return; Springt an den Anfang der Zeile.. 
* `\t`: Tabulator Whitespace.
* `\b`: ein einfacher Whitespace.
* `\u....`: definiert einen unicode codepoint (UTF-8) mittels hexadezimaler Zahlen.

In [4]:
print("But this won\'t")

[1] "But this won't"


In [5]:
print("\u01f6")

[1] "Ƕ"


# Text I/O

# Unicode

Im ursprünglichen ASCII encoding wurden Strings mittels 8bit repräsentiert, daher konnten nur 256 Zeichen dargestellt werden. Um die Bandbreite natürlicher Sprachen abzubilden wurde der Unicode Standard eingeführt. Mittels eines spezifischen Encodings (z.B.: UTF-8) wird die Korrespondenz eines bestimmten codepoints (unicode character) mit einem in einer gegebenen Schriftart darstellbaren Zeichen verknüpft.

Um das encoding eines Textstreams zu verändern kann die Funktion `file()` genutzt werden. Die Kodierung kann dann über das Argument `encoding="..."` angegeben werden.

In [31]:
lines <- readLines(file('../data/ucexample_Weber_utf8.txt', 
                        encoding='UTF-8'))

In [32]:
show(lines)

[1] "Der Nationalstaat und die Volkswirtschaftspolitik"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## Raw text input

R's standard function to read text files is `readLines()`. As the name suggests, it actually produces a vector of lines (seperated by newlines) from the file input. To preserve the formatting and get the *plain text* content, the function `readr::read_file()` can be used. Other packages offer additional read/write functions for text.

In [28]:
plain <- read_file('../data/ucexample_Weber_utf8.txt')

In [29]:
show(plain)

[1] "Der Nationalstaat und die Volkswirtschaftspolitik\n\n1895\nVorbemerkung\n\nNicht die Zustimmung, sondern der Widerspruch, welchen die nachstehenden Ausführungen bei vielen ihrer Hörer fanden, veranlaßten mich, sie zu veröffentlichen. Sachlich Neues werden sie Fachgenossen wie andern nur in Einzelheiten bringen, und in welchem speziellen Sinn allein sie den Anspruch auf das Prädikat der »Wissenschaftlichkeit« erheben, ergibt sich aus der Veranlassung ihres Entstehens. Eine Antrittsrede bietet eben Gelegenheit zur offenen Darlegung und Rechtfertigung des persönlichen und insoweit »subjektiven« Standpunktes bei der Beurteilung volkswirtschaftlicher Erscheinungen. Die Ausführungen S. 15 bis 18 (oben)hatte ich mit Rücksicht auf Zeit und Hörerkreis fortgelassen, andere mögen beim Sprechen eine andere Form angenommen haben. Zu den Darlegungen im Eingang ist zu bemerken, daß die Vorgänge hier naturgemäß wesentlich vereinfacht gegenüber der Wirklichkeit dargestellt werden. Die Zeit von 187

## CSV files

One of the most common and robust data types is CSV (*comma seperated values*). Each line contains a row of data, seperated by a specific character (e.g. comma). Optionally, the first line contains a header giving the column names.

In [9]:
econ_df <- as_tibble(read.csv(file="data//EconAbstRaw.csv", 
                              header=TRUE, 
                              sep=","))

In [10]:
dim(econ_df)

[1] 6556   10

In [10]:
names(econ_df)

[1] "X"                          "Authors"                   
 [3] "Year.of.Publication"        "Title"                     
 [5] "Author.Keywords"            "KeyWords.Plus"             
 [7] "Abstracts"                  "ISO.4.Journal.Abbreviation"
 [9] "Zeitschrift"                "Origin"

In [11]:
table(econ_df$Zeitschrift)


                        AMERICAN ECONOMIC REVIEW 
                                            1020 
     AMERICAN JOURNAL OF ECONOMICS AND SOCIOLOGY 
                                             587 
                      ECONOMIC AND SOCIAL REVIEW 
                                             158 
       INTERNATIONAL JOURNAL OF SOCIAL ECONOMICS 
                                              92 
                  JOURNAL OF ECONOMIC LITERATURE 
                                              58 
                  JOURNAL OF ECONOMIC PSYCHOLOGY 
                                             760 
                  JOURNAL OF FINANCIAL ECONOMICS 
                                            1157 
                    JOURNAL OF POLITICAL ECONOMY 
                                             782 
JOURNAL OF SOCIAL POLITICAL AND ECONOMIC STUDIES 
                                               7 
                  QUARTERLY JOURNAL OF ECONOMICS 
                                             753 

# String Operationen

## `stringr`

Dieses Paket stellt Standardfunktionen für die Manipulation von Strings bereit. Diese beginnen immer mit `str_`.

* `str_length()`: gibt Länge des Strings zurück; respektiert `NA`.
* `str_c()`: konkateniert Strings.
* `str_sub()`: Subsetting für Strings (funktioniert wie ein Index).
* `str_to_lower()`, `...upper()`, `...title()`: Gibt lower/upper/title case Repräsentation des Strings zurück.
* `str_sort()`: returns a sorted representation of the string.
* `str_extract()`: Extrahiert Substrings die dem angegeben Muster entsprechen.
* `str_detect()`: Gibt einen boolschen Vektor entsprechend des angegebenen Musters aus.
* `str_count()`: Zählt Häufigkeit des Vorkommens im String.

Manche dieser Funktionen lassen sich mit `_all` auf alle Übereinstimmungen mit einem Muster innerhalb des Strings verallgmeinern. Ansonsten wird immer nur die erste Übereinstimmung verwendet.

In [16]:
## <TAB> to expand
?str_

In [8]:
str_c("Ham, bacon", "Spam", sep=" and ")

[1] "Ham, bacon and Spam"

In [17]:
breakfast <- c("eggs", "bacon", "spam", "lobster")
breakfast

[1] "eggs"    "bacon"   "spam"    "lobster"

In [10]:
str_c(breakfast, collapse=", ")

[1] "eggs, bacon, spam, lobster"

In [20]:
str_c(breakfast, "NA")

[1] "eggsNA"    "baconNA"   "spamNA"    "lobsterNA"

In [21]:
## NA is contagious!
str_c(breakfast, NA)

[1] NA NA NA NA

## Vektorisierung

Die meisten `stringr`-Funktionen werden bei Anwendung auf einen Vektor automatisch auf alle Einzelelemente angewendet. Die Rückgabe ist in diesem Fall ein Vektor.

In [22]:
str_length('breakfast')

[1] 9

In [23]:
str_length(breakfast)

[1] 4 5 4 7

In [24]:
str_to_lower(breakfast)

[1] "eggs"    "bacon"   "spam"    "lobster"

## Locale

`stringr` Funktionen erlauben die Angabe eines Sprachraums mittels des entsprechenden [ISO 639 Codes](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes).

In [25]:
## Default english language ("en")
str_to_upper(c("i", "ı"))

[1] "I" "I"

In [26]:
## Turkish upper case
str_to_upper(c("i", "ı"), locale="tr")

[1] "İ" "I"

# Reguläre Ausdrücke

## Ein paar Worte der Warnung

> Some people, when confronted with a problem, think “I know, I'll use regular expressions.” Now they have two problems.

> *Jamie Zawinski* 	 

## Was sind RegEx

Spezifizieren eine Auswahl aus einem endlichen Zeichenvorrat (Alphabet) $\Sigma$. Im Prinzip handelt es sich um eine sehr elgante Art Teile von Zeichenketten mit Platzhaltern auszuwählen.

Sie stehen in fast allen Betriebssystemen und Programmiersprachen zur Verfügung (siehe [hier](https://www.regular-expressions.info/)).

## Formale Definition

Wenn $x$ und $y$ reguläre Ausdrücke sind, dann sind:

2. Verkettung: $(xy)$
1. Alternative: $(x|y)$
3. Wiederholung (Kleene-Stern): $(x^*)$

ebenfalls valide, reguläre Ausdrücke.

## Implementation

Zwei dominante Standards für Reguläre Ausdrücke:

* [POSIX ERE](http://www.regular-expressions.info/posix.html)
* [PCRE - Perl Compatible Regular Expressions](http://www.pcre.org/)

## ... und dann gibt es noch R

http://www.regular-expressions.info/rlanguage.html:

>  The R Project for Statistical Computing provides seven regular expression functions in its base package. The R documentation claims that the default flavor implements POSIX extended regular expressions. That is not correct. In R 2.10.0 and later, the default regex engine is a modified version of Ville Laurikari's TRE engine. It mimics POSIX but deviates from the standard in many subtle and not-so-subtle ways. What this website says about POSIX ERE does not (necessarily) apply to R.

## Funktionen

R-base hat wie  die meisten Programmiersprachen grundlegende Möglichkeiten zum Umgang mit RegEx (e.g. `grep`). Zusätzlich verstehen die meisten `stringr` or `stringi` Funktionen ebenfalls reguläre Ausdrücke.

## String matching

In der einfachsten Variante können reguläre Ausdrücke einfach zur Spezifikation eines Teils einer Zeichenkette genutzt werden. 

In [48]:
breakfast <- 'Egg and bacon\
Egg, sausage, and bacon\
Egg and Spam\
Egg, bacon, and Spam\
Egg, bacon, sausage, and Spam\
Spam, bacon, sausage, and Spam\
Spam, egg, Spam, Spam, bacon, and Spam\
Spam, Spam, Spam, egg, and Spam\
Spam, Spam, Spam, Spam, Spam, Spam, baked beans, Spam, Spam, Spam, and Spam\
Lobster Thermidor aux crevettes with a Mornay sauce, garnished with truffle pâté, brandy, and a fried egg on top, and Spam.'

In [50]:
str_extract(breakfast, 'Spam')

[1] "Spam"

In [51]:
str_extract_all(breakfast, 'Spam')

[[1]]
 [1] "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam"
[11] "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam" "Spam"
[21] "Spam" "Spam" "Spam" "Spam"

## Sonderzeichen

* `.` : entspricht arbiträrem Zeichen außer der Newline.
* `^` : entspricht dem Anfang eines Strings.
* `$` : entspricht dem Ende eines Strings.
* `\` : Escaping von Sonderzeichen.

In [54]:
str_extract_all(breakfast, ".gg.")

[[1]]
[1] "Egg " "Egg," "Egg " "Egg," "Egg," "egg," "egg," "egg "

## Alternative

Der `|` verhält sich ähnlich wie ein logisches ODER.

In [56]:
str_extract_all(breakfast, 'Egg|egg')

[[1]]
[1] "Egg" "Egg" "Egg" "Egg" "Egg" "egg" "egg" "egg"

In [44]:
dennis <- "Listen, strange women lying in ponds distributing swords is no basis for a system of government. Supreme executive power derives from a mandate from the masses, not from some farcical aquatic ceremony."
dennis

[1] "Listen, strange women lying in ponds distributing swords is no basis for a system of government. Supreme executive power derives from a mandate from the masses, not from some farcical aquatic ceremony."

In [46]:
alice <- '\'When I\'M a Duchess,\' she said to herself, (not in a very hopeful tone though), \'I won\'t have any pepper in my kitchen AT ALL. Soup does very well without--Maybe it\'s always pepper that makes people hot-tempered\'.'
alice

[1] "'When I'M a Duchess,' she said to herself, (not in a very hopeful tone though), 'I won't have any pepper in my kitchen AT ALL. Soup does very well without--Maybe it's always pepper that makes people hot-tempered'."

## Wiederholungen

Spezifiziert Anzahl der Wiederholungen des vorangegangenen regulären Ausdrucks $x$. Folgende Wiederholungen sind möglich:

Syntax | Bedeutung
-|-
`*` | 0 oder mehr Wiederholungen
`+` | 1 oder mehr Wiederholungen
`{m}` | Genau `m` Wiederholungen
`{m,n}` | Von `m` bis einschließlich `n`
`?` | 0 bis 1 Wiederholungen; Schaltet greedy ab.

Die Wiederholungen sind standardmäßig *greedy*, d.h. es wird soviel vom String verbraucht, wie möglich. Dieses Verhalten kann abgeschaltet werden, indem ein `?` nach der Wiederholung gesetzt wird.

In [57]:
## Greedy by default!
gene <- "GCUGCCGCAGCG"

str_extract(gene, "C.+C")

[1] "CUGCCGCAGC"

## Spezifizierung von Gruppen

Syntax | Äquivalent | Bedeutung
-|-|-
`\d` | `[0-9]` | Ganze Zahlen
`\D` | `[^0-9]` | Alles was keine Zahl ist
`\s` | `[ \t\n\r\f\v]` | Alles was whitespace ist 
`\S` | `[^ \t\n\r\f\v] ` | Alles was nicht whitespace ist
`\w` | `[a-zA-Z0-9_]` | Alphanumerische Zeichen und Unterstrich
`\W` | `[^a-zA-Z0-9_]` | Kein alphanumerische Zeichen oder Unterstrich

## Texte in Token zerlegen

In [60]:
tokens <- str_extract_all(alice, "\\w+[-\']?\\w+|\\S+?")
tokens

[[1]]
 [1] "'"            "When"         "I'M"          "a"            "Duchess"     
 [6] ","            "'"            "she"          "said"         "to"          
[11] "herself"      ","            "("            "not"          "in"          
[16] "a"            "very"         "hopeful"      "tone"         "though"      
[21] ")"            ","            "'"            "I"            "won't"       
[26] "have"         "any"          "pepper"       "in"           "my"          
[31] "kitchen"      "AT"           "ALL"          "."            "Soup"        
[36] "does"         "very"         "well"         "without"      "-"           
[41] "-"            "Maybe"        "it's"         "always"       "pepper"      
[46] "that"         "makes"        "people"       "hot-tempered" "'"           
[51] "."

## Eine valide Emailadresse finden

![Regular Expression](https://imgs.xkcd.com/comics/regular_expressions.png)

In der Wirklichkeit sehr viel Komplizierter: http://www.ex-parrot.com/~pdw/Mail-RFC822-Address.html.

# Anwendungsbeispiel

## Vokale zählen

In [61]:
str_count(breakfast, "[aeiou]")

[1] 95

## Suchen und ersetzen

In [76]:
bsp <- 'Ersetzen sie bei allen Zitaten die eckigen Klammern ([])\
mit den korrekten, runden Klammern.\
Spezifisch bei Blabla [2010] aber auch bei anderen [z.B.: Foobaz 2009, 17].'


str_replace_all(bsp, '\\[(.+?)\\]', '\\(\\1\\)')

[1] "Ersetzen sie bei allen Zitaten die eckigen Klammern ([])\nmit den korrekten, runden Klammern.\nSpezifisch bei Blabla (2010) aber auch bei anderen (z.B.: Foobaz 2009, 17)."

## HTML oder anderes Markup entfernen

In [79]:
## Remove the tags
htmltag <- "<p>text</p>"

str_replace_all(htmltag, "<.*?>", "")

[1] "text"